## Install and Import Dependencies

In [1]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [60]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

## Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

## Encode and Calculate Sentiment

In [50]:
tokens = tokenizer.encode("out of worst, this is best", return_tensors='pt')

In [51]:
result = model(tokens)

In [52]:
int(torch.argmax(result.logits))+1

5

## Collect Reviews

In [56]:
# Web Scrapper
r = requests.get("https://www.yelp.com/biz/mejico-sydney-2")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class': regex})
reviews = [result.text for result in results]

In [61]:
reviews[0]

'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!'

## Load reviews and DataFrame and Score

In [62]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [63]:
df.head()

,review
0,The food and service here was really good. It...
1,Visiting from Texas and decided to give this r...
2,"Great atmosphere, attentive service, solid mar..."
3,Don't come here expecting legit Mexican food b...
4,Out of all the restaurants that I tried in Syd...


In [64]:
df['review'].iloc[0]

'The food and service here was really good. \xa0It was more like tapas food than Mexican food! \xa0The drinks were amazing too!'

In [65]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [68]:
sentiment_score(df['review'].iloc[1])

5

In [69]:
df.shape

(10, 1)

In [70]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x[:512]))

In [71]:
df

,review,sentiment
0,The food and service here was really good. It...,5
1,Visiting from Texas and decided to give this r...,5
2,"Great atmosphere, attentive service, solid mar...",3
3,Don't come here expecting legit Mexican food b...,3
4,Out of all the restaurants that I tried in Syd...,5
5,We came here on a Thursday night @ 5pm and by ...,4
6,The food is fresh and tasty. The scallop cevi...,4
7,Have been here twice and have absolutely loved...,5
8,I was pleasantly surprised at what a great job...,5
9,If you're looking for a quiet little romantic ...,2
